# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [13]:
!pip install xlsxwriter
!pip install python-dotenv

import os
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

from dotenv import load_dotenv
load_dotenv()

True

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [14]:
pwd = '/content/trading_practice/algorithmic-trading-python/starter_files'

stocks = pd.read_csv(f'{pwd}/sp_500_stocks.csv')
print(len(stocks.index))

token = os.getenv("IEX_CLOUD_API_TOKEN")
print(token)

505
Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [15]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={token}'
data = requests.get(api_url).json()
data

{'avg10Volume': 122705751,
 'avg30Volume': 105541037,
 'beta': 1.1690099210016751,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 116.1,
 'day30ChangePercent': 0.13755936770161706,
 'day50MovingAvg': 124.44,
 'day5ChangePercent': 0.04388825950323173,
 'dividendYield': 0.00610374853991458,
 'employees': 142745,
 'exDividendDate': '2020-11-02',
 'float': 0,
 'marketcap': 2340406823694,
 'maxChangePercent': 49.9154829520143,
 'month1ChangePercent': 0.13972892773130516,
 'month3ChangePercent': 0.18105769835579044,
 'month6ChangePercent': 0.4711240894024029,
 'nextDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 39.143011018065785,
 'sharesOutstanding': 17168563993,
 'ttmDividendRate': 0.8251033452643882,
 'ttmEPS': 3.41,
 'week52change': 0.8779231612984505,
 'week52high': 140.31,
 'week52low': 57.77,
 'year1ChangePercent': 0.8864823311598857,
 'year2ChangePercent': 2.765092438923646,
 'year5ChangePercent': 4.312315745483525,
 'ytdChangePercent': 0.8479466378045913}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [16]:
data['year1ChangePercent']

0.8864823311598857

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [17]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [18]:
import json

df = pd.DataFrame(columns = my_columns)

batch_url = 'https://sandbox.iexapis.com/stable/stock/market/batch'

for symbol_string in symbol_strings:
    batch_api_call_url = f'{batch_url}/?types=stats,quote&symbols={symbol_string}&token={token}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ], 
                index = my_columns), 
                ignore_index = True
                )
        
    
df

,Ticker,Price,One-Year Price Return,Shares to Buy
0,A,117.78,0.397622,N/A
1,AAL,16.13,-0.472104,N/A
2,AAP,163.45,0.00856321,N/A
3,AAPL,137.63,0.906846,N/A
4,ABBV,105.44,0.218976,N/A
...,...,...,...,...
500,YUM,111.29,0.0967063,N/A
501,ZBH,151.50,-0.001265,N/A
502,ZBRA,379.31,0.488791,N/A
503,ZION,43.52,-0.128253,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [19]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
df = df[:50]

df = df.reset_index(level=0, drop=True)

df.head()

,Ticker,Price,One-Year Price Return,Shares to Buy
0,CARR,39.02,2.2324,N/A
1,PYPL,240.98,1.23026,N/A
2,LB,39.13,1.19489,N/A
3,NVDA,524.38,1.19146,N/A
4,ALB,154.99,1.13916,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [32]:
def portfolio_size():
    global p_size
    p_size = input('Enter the size of your portfolio: ')
    
    try:
        float(p_size)
    except ValueError:
        print("Not a number")
        portfolio_size()

portfolio_size()

In [38]:
position_size = float(p_size)/len(df)

for i in range(0, len(df)):
    df.loc[i, 'Shares to Buy'] = math.floor(position_size/df.loc[i, 'Price'])

df.head()

,Ticker,Price,One-Year Price Return,Shares to Buy
0,CARR,39.02,2.2324,5125
1,PYPL,240.98,1.23026,829
2,LB,39.13,1.19489,5111
3,NVDA,524.38,1.19146,381
4,ALB,154.99,1.13916,1290


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [46]:
hqm_cols = [
    'Ticker',
    'Price',
    'Shares to Buy',
    '1Y Price Return',
    '1Y Return Percentile',
    '6M Price Return',
    '6M Return Percentile',
    '3M Price Return',
    '3M Return Percentile',
    '1M Price Return',
    '1M Return Percentile',
]

hqm_df = pd.DataFrame(columns=hqm_cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'{batch_url}/?types=stats,quote&symbols={symbol_string}&token={token}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'NA',
                    data[symbol]['stats']['year1ChangePercent'],
                    'NA',
                    data[symbol]['stats']['month6ChangePercent'],
                    'NA',
                    data[symbol]['stats']['month3ChangePercent'],
                    'NA',
                    data[symbol]['stats']['month1ChangePercent'],
                    'NA',
                ], 
                index = hqm_cols), 
                ignore_index = True
                )
hqm_df

hange': 0, 'extendedChangePercent': 0, 'extendedPriceTime': 1657180783872, 'previousClose': 42.01, 'previousVolume': 19426022, 'change': -0.18, 'changePercent': -0.00415, 'volume': 8207943, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 29681492, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'iexOpen': 41.6, 'iexOpenTime': 1610375001005, 'iexClose': 41.8, 'iexCloseTime': 1664595966572, 'marketCap': 178745418775, 'peRatio': 55.7, 'week52High': 67.79, 'week52Low': 30.88, 'ytdChange': -0.3728050132681246, 'lastTradeTime': 1648914170503, 'isUSMarketOpen': False}}, 'XRAY': {'stats': {'companyName': 'DENTSPLY Sirona Inc', 'marketcap': 11326664224, 'week52high': 62.29, 'week52low': 31.8, 'week52change': -0.08175971743189035, 'sharesOutstanding': 225233391, 'float': 0, 'avg10Volume': 1172787, 'avg30Volume': 1134671, 'day200MovingAvg': 47.58, 'day50MovingAvg': 53.55, 'employees': 16462, 'ttmEPS': -0.2199, 'ttmDividendRate': 0.4178697112358935,

,Ticker,Price,Shares to Buy,1Y Price Return,1Y Return Percentile,6M Price Return,6M Return Percentile,3M Price Return,3M Return Percentile,1M Price Return,1M Return Percentile
0,A,122.37,NA,0.398599,NA,0.359864,NA,0.211716,NA,0.0267832,NA
1,AAL,15.99,NA,-0.474246,NA,0.18916,NA,0.284954,NA,0.0501126,NA
2,AAP,168.07,NA,0.00879266,NA,0.129342,NA,0.0790237,NA,0.0796205,NA
3,AAPL,137.05,NA,0.887205,NA,0.460069,NA,0.178456,NA,0.144205,NA
4,ABBV,108.02,NA,0.219329,NA,0.0964751,NA,0.223404,NA,-0.00937534,NA
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.86,NA,0.0944597,NA,0.267005,NA,0.189381,NA,0.00882999,NA
501,ZBH,149.27,NA,-0.00131228,NA,0.311745,NA,0.11319,NA,-0.00127677,NA
502,ZBRA,385.59,NA,0.486665,NA,0.499213,NA,0.484561,NA,0.0114873,NA
503,ZION,43.75,NA,-0.132809,NA,0.301302,NA,0.538528,NA,0.0584747,NA


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: